<h1>Lymphocyte Gating</h1>

Testing the performance of Immunova's automated gating algorithms at detecting lymphocyte populations gated using forward and sideward scatter. The accuracy, sensitivity, specificity and F1 score will be reported for each sample and then summarised as the average across all samples.

In [1]:
import sys
if '/home/rossco/immunova' not in sys.path:
    sys.path.append('/home/rossco/immunova')
from immunova.data.fcs_experiments import FCSExperiment
from immunova.flow.gating.actions import Gating
from immunova.flow.plotting.static_plots import Plot
from immunova.data.mongo_setup import test_init
import warnings
warnings.filterwarnings('ignore')
test_init()

/home/rossco/immunova/env/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/rossco/immunova/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
exp = FCSExperiment.objects(experiment_id='ILTIS_T1').get()

In [3]:
sep6 = Gating(exp, 'sep6')

{'id': 'sep6', 'typ': 'complete', 'data':                  FSC-A     FSC-H          FSC-W          SSC-A     SSC-H  \
0        200913.296875  117800.0  111774.656250  262143.000000  203276.0   
1        219078.718750  168977.0   84967.445312  262143.000000  221489.0   
2        133687.265625  103241.0   84862.875000   83807.281250   74505.0   
3        120320.546875   83503.0   94431.664062  256666.000000  215355.0   
4        125345.429688   97907.0   83902.460938  262143.000000  215539.0   
...                ...       ...            ...            ...       ...   
1004933   92054.335938   74214.0   81290.234375   64599.320312   60786.0   
1004934  129530.515625   66932.0  126828.906250  262143.000000  237588.0   
1004935   77889.421875   64022.0   79731.359375   30483.041016   28790.0   
1004936  157190.031250  109667.0   93935.320312  262143.000000  230883.0   
1004937  111750.656250   80499.0   90978.664062  262143.000000  234167.0   

                SSC-W         CD57       CD16

AttributeError: 'dict' object has no attribute 'index'